In [ ]:
!pip install tensorflow==2.12.0
!pip install ampligraph

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np

import ampligraph
import numpy as np
import pandas as pd
import tensorflow as tf
from ampligraph.datasets import load_fb15k_237
from ampligraph.evaluation.protocol import train_test_split_no_unseen
from ampligraph.evaluation.metrics import mr_score, mrr_score, hits_at_n_score

from ampligraph.datasets import load_wn18
from ampligraph.latent_features import ScoringBasedEmbeddingModel
from ampligraph.evaluation import mrr_score, hits_at_n_score
from ampligraph.latent_features.loss_functions import get as get_loss
from ampligraph.latent_features.regularizers import get as get_regularizer
import tensorflow as tf

from ampligraph.latent_features.layers.scoring import TransE, ComplEx, HolE, DistMult
from ampligraph.utils import save_model, restore_model
from ampligraph.evaluation import select_best_model_ranking
from ampligraph.latent_features.models.ScoringBasedEmbeddingModel import ScoringBasedEmbeddingModel
from ampligraph.utils import save_model, restore_model
import gradio as gr
import tensorflow as tf

print('TensorFlow  version: {}'.format(tf.__version__))

In [ ]:
ComplEx_model_path = '/content/drive/MyDrive/kg_models/ComplEx.pkl'
RotatE_model_path = '/content/drive/MyDrive/kg_models/RotatE.pkl'
TransE_model_path = '/content/drive/MyDrive/kg_models/TransE.pkl'

restored_ComplEx_model = restore_model(model_name_path=ComplEx_model_path)
restored_RotatE_model = restore_model(model_name_path=RotatE_model_path)
restored_TransE_model = restore_model(model_name_path=TransE_model_path)

df=pd.read_csv('all_triplets.csv',index_col=False)

In [ ]:
test_train, X_valid = train_test_split_no_unseen(df.values, 500, seed=0)
X_train, X_test = train_test_split_no_unseen(test_train, 1000, seed=0)

print('Total triples:', df.shape)
print('Size of train:', X_train.shape)
print('Size of valid:', X_valid.shape)
print('Size of test:', X_test.shape)

X = {
    'train': X_train,
    'valid': X_valid,
    'test':  X_test,
}

filter_dict = {'test': np.concatenate((X['train'], X['valid'], X['test']))}

In [ ]:
def evaluate_model(model, test_data, filter_dict):
    ranks = model.evaluate(test_data, use_filter=filter_dict, corrupt_side='s,o')
    return {
        "MRR": mrr_score(ranks),
        "Hits@1": hits_at_n_score(ranks, n=1),
        "Hits@10": hits_at_n_score(ranks, n=10),
        "Hits@50": hits_at_n_score(ranks, n=50),
        "Hits@100": hits_at_n_score(ranks, n=100)
    }
results = {
    "ComplEx": evaluate_model(restored_ComplEx_model, X['test'], filter_dict),
    "RotatE": evaluate_model(restored_RotatE_model, X['test'], filter_dict),
    "TransE": evaluate_model(restored_TransE_model, X['test'], filter_dict)
}

comparison_df = pd.DataFrame(results).T.reset_index().rename(columns={"index": "Model"})
comparison_df

In [ ]:
# Prepare random test subset
np.random.seed(98)
testing_subset = X['test'][np.random.choice(len(X['test']), size=10, replace=False)]
entities = sorted(set(df['subject']) | set(df['object']))
def get_tail_prediction_ranks_all_models(test_triples, models_dict, entities, filter_dict, top_k=10):
    results = []
    known_triples = set(tuple(triple) for triple in filter_dict['test'])

    for h, r, true_t in test_triples:
        row = {
            "Head": h,
            "Relation": r,
            "True Tail": true_t
        }

        for model_name, model in models_dict.items():
            candidate_triplets = [(h, r, t) for t in entities]
            filtered_candidates = [
                (h_tmp, r_tmp, t_tmp)
                for (h_tmp, r_tmp, t_tmp) in candidate_triplets
                if (h_tmp, r_tmp, t_tmp) not in known_triples or t_tmp == true_t
            ]

            scores = model.predict(filtered_candidates)
            sorted_indices = np.argsort(scores)[::-1]
            ranked_tails = [filtered_candidates[i][2] for i in sorted_indices]
            rank = ranked_tails.index(true_t) + 1 if true_t in ranked_tails else "Not in top"

            row[f"Rank ({model_name})"] = rank

        results.append(row)

    return pd.DataFrame(results)

models_dict = {
    "ComplEx": restored_ComplEx_model,
    "RotatE": restored_RotatE_model,
    "TransE": restored_TransE_model
}

df_multi_rank = get_tail_prediction_ranks_all_models(testing_subset, models_dict, entities, filter_dict, top_k=10)
df_multi_rank